In [1]:
!pip install tensorflow-hub
!pip install numpy==1.16.1
#!pip install keras==2.1.2
!pip install git+https://github.com/LanceNorskog/deep_meter_2#egg=deepmeter

from __future__ import print_function
import math
import pickle
import json

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import keras.backend as K
import matplotlib.pyplot as plt

from keras.preprocessing import sequence
from keras.models import Model, Sequential
from keras import layers 
from keras import metrics
from keras.datasets import reuters
from keras.preprocessing import text

from sklearn.model_selection import train_test_split

from cmu.syllables_cmu import syllables as word2sylls
print(word2sylls['therefore'])

# cut texts after this number of words
# (among top max_features most common words)
max_features = 2000
# longest output sentence
maxlen = 5
batch_size = 32

!wget -nc https://raw.githubusercontent.com/LanceNorskog/deep_meter_2/master/haiku_5_same.txt
haiku_data = 'haiku_5_same.txt'
haiku_text = []
with open(haiku_data) as f:
    for line in f.readlines():
        haiku_text.append(line.split('\t')[0])


['DH EH R', 'F AO R']


Using TensorFlow backend.


File ‘haiku_5_same.txt’ already there; not retrieving.



In [0]:
def tokenizer_from_json(json_string):
    """Parses a JSON tokenizer configuration file and returns a
    tokenizer instance.
    # Arguments
        json_string: JSON string encoding a tokenizer configuration.
    # Returns
        A Keras Tokenizer instance
    """
    tokenizer_config = json.loads(json_string)
    config = tokenizer_config.get('config')

    word_counts = json.loads(config.pop('word_counts'))
    word_docs = json.loads(config.pop('word_docs'))
    index_docs = json.loads(config.pop('index_docs'))
    # Integer indexing gets converted to strings with json.dumps()
    index_docs = {int(k): v for k, v in index_docs.items()}
    index_word = json.loads(config.pop('index_word'))
    index_word = {int(k): v for k, v in index_word.items()}
    word_index = json.loads(config.pop('word_index'))

    tokenizer = text.Tokenizer(**config)
    tokenizer.word_counts = word_counts
    tokenizer.word_docs = word_docs
    tokenizer.index_docs = index_docs
    tokenizer.word_index = word_index
    tokenizer.index_word = index_word

    return tokenizer

In [3]:
!date
!wget -nc https://raw.githubusercontent.com/LanceNorskog/deep_meter_2/master/haiku_5_same.txt
!wget -nc https://raw.githubusercontent.com/LanceNorskog/deep_meter_2/master/toki_cmu.json
haiku_data = 'haiku_5_same.txt'
toki_file = 'toki_cmu.json'
haiku_text = []
with open(haiku_data) as f:
    for line in f.readlines():
        haiku_text.append(line.split('\t')[0])
!date
print('haiku_text: ', len(haiku_text))
print('one: ', haiku_text[0])
with open(toki_file, "r") as f:
        conf = f.read()
        toki = tokenizer_from_json(conf)
        conf = None
!date
data_all = [[]] * len(haiku_text)
for i in range(len(haiku_text)):    
    seq = text.text_to_word_sequence(haiku_text[i])
    sylls = []
    for word in seq:
        if word in word2sylls:
            for syll in word2sylls[word]:
                sylls.append(syll)
    data_all[i] = toki.texts_to_sequences(sylls)

!date   
print(haiku_text[0])
print(data_all[0])
print(haiku_text[2999])
print(data_all[2999])
!date
#(x_train, x_test, y_train, y_test) = train_test_split(haiku_text, data_all)


Wed Jun 26 05:14:18 UTC 2019
File ‘haiku_5_same.txt’ already there; not retrieving.

File ‘toki_cmu.json’ already there; not retrieving.

Wed Jun 26 05:14:21 UTC 2019
haiku_text:  95622
one:  a abandon church
Wed Jun 26 05:14:21 UTC 2019
Wed Jun 26 05:14:30 UTC 2019
a abandon church
[[1], [1], [11, 10, 3], [8, 1, 3], [32, 19, 32]]
a cardboard cutout
[[1], [9, 14, 5, 8], [11, 25, 5, 8], [9, 1, 2], [31, 2]]
Wed Jun 26 05:14:30 UTC 2019


In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
embed = hub.Module(module_url)
embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]



In [5]:
(x_train, x_test, y_train, y_test) = train_test_split(haiku_text, data_all)
print(x_train[0][0], y_train[0][0])
print(x_test[0][0], y_test[0][0])

def get_lstm(size, return_sequences=True):
    return layers.CuDNNLSTM(size, return_sequences=return_sequences)

def pack(indexes, maxlen):
    print(type(indexes))
    print(type(indexes[0]))
    print(type(indexes[0][0]))
    print(len(indexes[0][0]))
    ilen = len(indexes)
    out = np.zeros((ilen, maxlen))
    for i in range(ilen):
        for j in range(len(indexes[i])):
            if j >= maxlen:
                continue
            #print(i,j, len(indexes[i]))  
            if len(indexes[i][j]) > 0:
                out[i][j] = indexes[i][j][0]
    return out
    
print('Pad sequences (samples x time)')
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.expand_dims(pack(y_train, maxlen), -1)
y_test = np.expand_dims(pack(y_test, maxlen), -1)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
print(y_test[0][0])



a [1]
h [30, 28, 7]
Pad sequences (samples x time)
<class 'list'>
<class 'list'>
<class 'list'>
1
<class 'list'>
<class 'list'>
<class 'list'>
3
x_train shape: (71716,)
x_test shape: (23906,)
y_train shape: (71716, 5, 1)
y_test shape: (23906, 5, 1)
[30.]


In [0]:
# https://github.com/philipperemy/keras-snail-attention/blob/master/attention.py
class AttentionBlock(layers.Layer):

    def __init__(self, dims, k_size, v_size, seq_len=None, **kwargs):
        self.k_size = k_size
        self.seq_len = seq_len
        self.v_size = v_size
        self.dims = dims
        self.sqrt_k = math.sqrt(k_size)
        self.keys_fc = None
        self.queries_fc = None
        self.values_fc = None
        super(AttentionBlock, self).__init__(**kwargs)

    def build(self, input_shape):
        # https://stackoverflow.com/questions/54194724/how-to-use-keras-layers-in-custom-keras-layer
        self.keys_fc = layers.Dense(self.k_size)
        self.keys_fc.build((None, self.dims))
        self._trainable_weights.extend(self.keys_fc.trainable_weights)

        self.queries_fc = layers.Dense(self.k_size)
        self.queries_fc.build((None, self.dims))
        self._trainable_weights.extend(self.queries_fc.trainable_weights)

        self.values_fc = layers.Dense(self.v_size)
        self.values_fc.build((None, self.dims))
        self._trainable_weights.extend(self.values_fc.trainable_weights)

    def call(self, inputs, **kwargs):
        # check that the implementation matches exactly py torch.
        keys = self.keys_fc(inputs)
        queries = self.queries_fc(inputs)
        values = self.values_fc(inputs)
        logits = K.batch_dot(queries, K.permute_dimensions(keys, (0, 2, 1)))
        mask = K.ones_like(logits) * np.triu((-np.inf) * np.ones(logits.shape.as_list()[1:]), k=1)
        logits = mask + logits
        probs = layers.Softmax(axis=-1)(logits / self.sqrt_k)
        read = K.batch_dot(probs, values)
        output = K.concatenate([inputs, read], axis=-1)
        return output

    def compute_output_shape(self, input_shape):
        output_shape = list(input_shape)
        output_shape[-1] += self.v_size
        return tuple(output_shape)



In [0]:
def sparse_categorical_accuracy(y_true, y_pred):
    # reshape in case it's in shape (num_samples, 1) instead of (num_samples,)
    if K.ndim(y_true) == K.ndim(y_pred):
        y_true = K.squeeze(y_true, -1)
    # convert dense predictions to labels
    y_pred_labels = K.argmax(y_pred, axis=-1)
    y_pred_labels = K.cast(y_pred_labels, K.floatx())
    return K.cast(K.equal(y_true, y_pred_labels), K.floatx())

def sparse_categorical_accuracy_per_sequence(y_true, y_pred):
    # reshape in case it's in shape (num_samples, 1) instead of (num_samples,)
    if K.ndim(y_true) == K.ndim(y_pred):
        y_true = K.squeeze(y_true, -1)
    # convert dense predictions to labels
    y_pred_labels = K.argmax(y_pred, axis=-1)
    y_pred_labels = K.cast(y_pred_labels, K.floatx())
    return K.min(K.cast(K.equal(y_true, y_pred_labels), K.floatx()), axis=-1)

def sparse_temporal_top_k_categorical_accuracy(y_true, y_pred, k=5):
    original_shape = K.shape(y_true)
    y_true = K.reshape(y_true, (-1, K.shape(y_true)[-1]))
    y_pred = K.reshape(y_pred, (-1, K.shape(y_pred)[-1]))
    top_k = K.in_top_k(y_pred, K.cast(K.max(y_true, axis=-1), 'int32'), k)
    return K.reshape(top_k, original_shape[:-1])

def sparse_temporal_top_k_categorical_accuracy_per_sequence(y_true, y_pred, k=5):
    original_shape = K.shape(y_true)
    y_true = K.reshape(y_true, (-1, K.shape(y_true)[-1]))
    y_pred = K.reshape(y_pred, (-1, K.shape(y_pred)[-1]))
    top_k = K.in_top_k(y_pred, K.cast(K.max(y_true, axis=-1), 'int32'), k)
    perfect = K.min(K.cast(top_k, 'int32'), axis=-1)
    return perfect #K.expand_dims(perfect, axis=-1)

def sparse(y_true, y_pred):
    return sparse_categorical_accuracy(y_true, y_pred)
def sparse1(y_true, y_pred):
    return sparse_temporal_top_k_categorical_accuracy(y_true, y_pred, k=1)
def perfect(y_true, y_pred):
    return sparse_categorical_accuracy_per_sequence(y_true, y_pred)
def perfect1(y_true, y_pred):
    return sparse_temporal_top_k_categorical_accuracy_per_sequence(y_true, y_pred, k=1)
def sparse5(y_true, y_pred):
    return sparse_temporal_top_k_categorical_accuracy(y_true, y_pred, k=5)
def perfect5(y_true, y_pred):
    return sparse_temporal_top_k_categorical_accuracy_per_sequence(y_true, y_pred, k=5)
def fscore(y_true, y_pred):
    recall = K.mean(sparse_categorical_accuracy(y_true, y_pred))
    precision = K.mean(sparse_categorical_accuracy_per_sequence(y_true, y_pred))
    return 2 * ((recall * precision)/(recall + precision))


units_k=embed_size
units_v=embed_size//3
units=1024

metric_list = [sparse, sparse1, perfect, perfect1, sparse5, perfect5]
metric_names = ['sparse', 'sparse1', 'perfect', 'perfect1', 'sparse5', 'perfect5']

input_text = layers.Input(shape=(1,), dtype=tf.string)
x = layers.Lambda(UniversalEmbedding, output_shape=(embed_size,), name='TF-Hub')(input_text)
x = layers.RepeatVector(maxlen)(x)
if False:
    #x = layers.Dropout(0.1)(x)
    x = AttentionBlock(embed_size, k_size=units_k, v_size=units_v)(x)
x = get_lstm(units, return_sequences=True)(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(max_features, activation='softmax')(x)
#x = layers.Reshape((-1, max_features * maxlen))(x)


model = Model(inputs=[input_text], outputs=x)
model.compile('adam', 'sparse_categorical_crossentropy', metrics=metric_list)
model.summary()

# try using different optimizers and different optimizer configs

print('Train...')
history = None
use_saved_model=False
if not use_saved_model or not os.path.exists('./model.h5'):
  with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())

    history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=30,
          verbose=2,
          validation_data=[x_test, y_test])
    model.save_weights('./model.h5')


W0626 05:14:33.827944 140564896233344 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0626 05:14:33.830021 140564896233344 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0626 05:14:35.606917 140564896233344 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0626 05:14:38.980397 140564896233344 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0626 05:14:38.993509 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
TF-Hub (Lambda)              (None, 512)               0         
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 5, 512)            0         
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 5, 1024)           6299648   
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 1024)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 5, 2000)           2050000   
Total params: 8,349,648
Trainable params: 8,349,648
Non-trainable params: 0
_________________________________________________________________


W0626 05:14:42.364511 140564896233344 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 71716 samples, validate on 23906 samples
Epoch 1/30
 - 149s - loss: 2.5075 - sparse: 0.3035 - sparse1: 0.3035 - perfect: 0.0051 - perfect1: 0.0000e+00 - sparse5: 0.6237 - perfect5: 0.0000e+00 - val_loss: 2.0752 - val_sparse: 0.4110 - val_sparse1: 0.4110 - val_perfect: 0.0207 - val_perfect1: 0.0000e+00 - val_sparse5: 0.7290 - val_perfect5: 0.0000e+00
Epoch 2/30


In [0]:

if history != None:
  # summarize history for accuracy
  for m in metric_names:
      plt.plot(history.history['val_' + m])
  plt.title('model accuracy')
  plt.xlabel('epoch')
  plt.legend(metric_names, loc='upper right')
  plt.show()

In [0]:
x_short = x_test[0:1000]
y_short = y_test[0:1000]
predicts = None

with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights('./model.h5')  
  #eval = model.evaluate(x_test, y_test)
  #print('model.evaluate on val holdout: ' ,model.metrics_names, eval)
  print('history: ', history)
  predicts = model.predict(x_short, batch_size=32)
  print('shape: {}'.format(predicts.shape))

print(len(predicts[0]))
print(len(predicts[0][0]))
print(predicts[0])

In [0]:
def maxindx(pred):
    maxi=-1
    maxv=-1.0
    for x in range(len(pred)):
        if pred[x] > maxv:
            maxv = pred[x]
            maxi = x
    return (maxi, maxv)

def match(data, prediction):
    good = 0
    total = 0
    for i in range(len(data)):
        #print(data[i], np.argmax(prediction[i]))
        if data[i] == np.argmax(prediction[i]):
            good += 1
        total += 1
    #print('{}, {}'.format(good, total))
    if (total == 0):
        return 0
    return good / total

parallel = 0.0
serial = 0.0
total = 0
for n in range(len(y_short)):
    #print(len(short[n]))
    check = match(y_short[n], predicts[n])
    parallel += check
    if check > 0.9999:
        serial += 1
    total += 1

print('Parallel, serial: ', parallel / total, serial / total)
